In [6]:
import numpy as np
import scipy
from matplotlib import pyplot as plt

In [7]:
# QSP Phase Finding, determined through optimize_phi(poly, deg_poly)
# Optimizes toward the +,+ component of the QSP sequence

def W(a): # x-rotation
    return np.array([[a, 1j*np.sqrt(1-a**2)], [1j*np.sqrt(1-a**2), a]])

def S(phi): # z-rotation
    return np.array([[np.exp(1j*phi), 0], [0, np.exp(-1j*phi)]])

def U(phi_array, a): # phi_array = horizontal array of phi input values
    d = len(phi_array)-1
    unitary = S(phi_array[0])
    for i in range(1, d+1):
        unitary = np.matmul(unitary, W(a))
        unitary = np.matmul(unitary, S(phi_array[i]))
    return unitary

def optimize_phi(poly, deg_poly): # optimizer to find QSP phases
    phi_array = np.random.rand(deg_poly+1)
    poly = polynomialize(poly)
    def objective(phi_array):
        loss = 0
        times = np.arange(-1,1,0.01)
        for a in times:
            estimate = np.real(U(phi_array,a)[0][0])+1j*np.real(U(phi_array,a)[0][1])
            loss += np.absolute(estimate - poly(a))**2
        return loss
    res = scipy.optimize.minimize(objective, phi_array)
    return res.x

In [8]:
def even_positive_checker(P): # checks P is even and positive definite
    if len(P)%2 == 0:
        return "Not Even"
    roots = np.roots(P)
    for root in roots:
        if np.isreal(root):
            counter = 0
            for r in roots:
                if r == root:
                    counter += 1
            if counter%2 == 1: # checks that all real roots have even multiplicity
                return "Not Positive Definite"

def roots_paired(P): # pairs roots with its complex conjugate
    roots = np.roots(P)
    real_roots = np.array([])
    for root in roots:
        if np.isreal(root):
            real_roots = np.append(real_roots,root) # all real roots
    real_roots = sorted(real_roots)
    paired = []
    for i in range(int(len(real_roots)/2)):
        if real_roots[2*i] == real_roots[2*i+1]:
            paired.append([real_roots[2*i],real_roots[2*i+1]]) # pairs the real roots
    complex_roots = np.array([])
    for root in roots:
        if not np.isreal(root):
            if np.imag(root)<0:
                root = np.conj(root)
            complex_roots = np.append(complex_roots,root)
    for i in range(int(len(complex_roots)/2)):
        if complex_roots[2*i] == complex_roots[2*i+1]:
            paired.append([complex_roots[2*i],np.conj(complex_roots[2*i+1])]) # pairs the complex roots
    return paired

def polynomialize(poly): # turns a set of poly coefficients into a function
    d = len(poly)-1
    return lambda x : np.sum([poly[i]*(x**(d-i)) for i in range(d+1)])

def max_abs_finder(P): # finds maximum magnitude of P on the range [-1,1]
    P = polynomialize(P)
    x_vals = np.arange(-1,1,0.01)    
    value = []
    
    for x in x_vals:
        value.append(P(x))
    pos = np.max(np.absolute(value))

    return pos

def mult(roots): # roots = list of roots, mult gives polynomial with given roots
    poly = [1]
    for root in roots:
        poly = np.polymul(poly, [1, -root])
    
    return poly

def separate_polynomial(poly): # turns polynomial into four constituent polynomials
    # Initialize empty lists for the four parts
    R_ER = np.zeros(len(poly), dtype=float)
    R_EI = np.zeros(len(poly), dtype=float)
    R_OR = np.zeros(len(poly), dtype=float)
    R_OI = np.zeros(len(poly), dtype=float)

    # iterate through the coefficients
    for i, p in enumerate(np.flip(poly)):
        real_part = p.real
        imag_part = p.imag
        
        if i % 2 == 0:  # even index
            R_ER[i] = real_part
            R_EI[i] = imag_part
        else:  # odd index
            R_OR[i] = real_part
            R_OI[i] = imag_part

    return np.flip(R_ER), np.flip(R_EI), np.flip(R_OR), np.flip(R_OI)
    
            
def factor(P,k): # factors P into product of k polynomials
    lead_coef = P[0]
    P = [i/P[0] for i in P] # make lead coef = 1
    P_2 = P[:-k] # find P_2, the parallelized polynomial
    print("P_2: " + str(P_2))
    
    if even_positive_checker(P_2) == "Not Even":
        return "Not Even"
    if even_positive_checker(P_2) == "Not Positive Definite":
        return "Not Positive Definite"
    
    paired = roots_paired(P_2)
    first_roots = [pair[0] for pair in paired]
    
    poly_roots = [[] for _ in range(k)]
    i = 0
    for root in first_roots:
        poly_roots[i%k].append(root) # set of roots for each R_j(x) polynomial
        i += 1
    print("P_2 Roots: "+str(poly_roots))
    print()
    factored_polys = []
    for root_list in poly_roots:
        factored_polys.append(mult(root_list))
    return factored_polys # coefficients for R_j(x) polynomials

# given poly P, finds the QSP phases for each of the four decomp polys of its factored polys
def QSP_phase_finder_all(P,k): 
    print("Input Polynomial: " + str(P))
    print()
    factored_polys = factor(P,k)
    
    if factored_polys == "Not Even":
        return "Not Even"
    if factored_polys == "Not Positive Definite":
        return "Not Positive Definite"
    
    print("Factored Polynomials:" + str(factored_polys))
    print()
    QSP_phases = {}
    for i in range(len(factored_polys)):
        poly = factored_polys[i]
        print("Factored Polynomial " + str(i+1) + ": " + str(poly))
        R_ER, R_EI, R_OR, R_OI = separate_polynomial(poly)
        print("R_ER: "+str(R_ER))
        print("R_EI: "+str(R_EI))
        print("R_OR: "+str(R_OR))
        print("R_OI: "+str(R_OI))
        print()
        
        scaling_ER = max(1,max_abs_finder(R_ER))
        scaling_EI = max(1,max_abs_finder(R_EI))
        scaling_OR = max(1,max_abs_finder(R_OR))
        scaling_OI = max(1,max_abs_finder(R_OI))
        
        R_ER = R_ER/(max(1,max_abs_finder(R_ER)))# bounds by 1 in magnitude
        R_EI = R_EI/(max(1,max_abs_finder(R_EI)))# bounds by 1 in magnitude
        R_OR = R_OR/(max(1,max_abs_finder(R_OR)))# bounds by 1 in magnitude
        R_OI = R_OI/(max(1,max_abs_finder(R_OI)))# bounds by 1 in magnitude
        
        
        phases = []
        phases.append(optimize_phi(R_ER, len(R_ER)-1))
        phases.append(optimize_phi(R_EI, len(R_EI)-1))
        phases.append(optimize_phi(R_OR, len(R_OR)-1))
        phases.append(optimize_phi(R_OI, len(R_OI)-1))
        
        QSP_phases["Factored Polynomial " + str(i+1) + " Phases"] = phases
        
    print("QSP phases: " + str(QSP_phases))
    print()
    
    
    return QSP_phases
